In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df = pd.read_csv('shark_clean.csv')

In [4]:
df.head()

,Unnamed: 0,Date,Year,Type,Country,Area,Location,Activity,Sex,Injury,Fatal (Y/N),Time,Species
0,0,09-Oct-2020,2020.0,Unprovoked,AUSTRALIA,Western Australia,Kelp Beds Beach (Kelpies),Surfing,M,Fatal,Y,10h45,Not Available
1,1,07-Oct-2020,2020.0,Unprovoked,USA,Florida,"Miami Beach, Miami-Dade County",Body Surfing,M,Survived,N,Afternoon,Blacktip shark
2,2,04-Oct-2020,2020.0,Unprovoked,AUSTRALIA,Western Australia,"Toms Surf break, Hamersley Pool, North Beach, ...",Surfing,M,No Injury,N,10h45,Bronze whaler 1.5m
3,4,20-Sep-2020,2020.0,Unprovoked,USA,Florida,"Sombero Key Light, Monroe County",Snorkeling,M,Survived,N,10h30,Bull shark
4,5,17-Sep-2020,2020.0,Provoked,AUSTRALIA,Queensland,Fraser Island,Fishing,M,Survived,N,16h00,"""whitetip shark"""


In [5]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [7]:
for i in df:
    print(i, '/n')
    
    print(df[i].value_counts(normalize=True).head())
    print('-----------------')

Date /n
1957    0.001528
1941    0.001189
1950    0.001189
1958    0.001189
1942    0.001189
Name: Date, dtype: float64
-----------------
Year /n
2015.0    0.022075
2017.0    0.021056
2016.0    0.020207
0.0       0.020207
2011.0    0.020037
Name: Year, dtype: float64
-----------------
Type /n
Unprovoked      0.797928
Provoked        0.099677
Watercraft      0.056546
Sea Disaster    0.038377
Invalid         0.004245
Name: Type, dtype: float64
-----------------
Country /n
USA                 0.365597
AUSTRALIA           0.213619
SOUTH AFRICA        0.088130
PAPUA NEW GUINEA    0.022075
NEW ZEALAND         0.021056
Name: Country, dtype: float64
-----------------
Area /n
Florida            0.173374
New South Wales    0.076414
Not Available      0.063678
Queensland         0.051622
Hawaii             0.047207
Name: Area, dtype: float64
-----------------
Location /n
Not Available                       0.075565
New Smyrna Beach, Volusia County    0.030056
Daytona Beach, Volusia County       0

# Date Categoricals

In [8]:
df['Year'] = [i[-4:] for i in df['Date']]

In [9]:
for i, d in enumerate(df['Year']):
    try:
        df['Year'][i] = int(d)
    except:
        df['Year'][i] = 0

In [10]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [11]:
df['Month'] = 'NA'

In [12]:
for i, d in enumerate(df['Date']):
    for m in months:
        if d.find(m) != -1:
            s = d.find(m)
            df['Month'][i] = d[s:s+3]

In [14]:
m = pd.get_dummies(df['Month'])

In [19]:
df = pd.concat([df,m], axis=1)
df.drop(['Date', 'NA'], axis=1, inplace=True)

# Type categoricals

In [22]:
def make_dummy(col):
    top_4_values = df[col].value_counts()[:4]
    other_values = df[col].value_counts()[4:]
    
    for name in top_4_values.keys():
        df.loc[df[col] == name, f'{col}: {name}'] =1
        
    for name in other_values.keys():
        df.loc[df[col] == name, f'{col}: other'] =1
    
    df.drop(col, axis=1, inplace=True)

In [23]:
make_dummy('Type')

In [24]:
df.head()

,Year,Country,Area,Location,Activity,Sex,Injury,Fatal (Y/N),Time,Species,Month,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Type: Unprovoked,Type: Provoked,Type: Watercraft,Type: Sea Disaster,Type: other
0,2020,AUSTRALIA,Western Australia,Kelp Beds Beach (Kelpies),Surfing,M,Fatal,Y,10h45,Not Available,Oct,0,0,0,0,0,0,0,0,0,0,1,0,1.0,NaN,NaN,NaN,NaN
1,2020,USA,Florida,"Miami Beach, Miami-Dade County",Body Surfing,M,Survived,N,Afternoon,Blacktip shark,Oct,0,0,0,0,0,0,0,0,0,0,1,0,1.0,NaN,NaN,NaN,NaN
2,2020,AUSTRALIA,Western Australia,"Toms Surf break, Hamersley Pool, North Beach, ...",Surfing,M,No Injury,N,10h45,Bronze whaler 1.5m,Oct,0,0,0,0,0,0,0,0,0,0,1,0,1.0,NaN,NaN,NaN,NaN
3,2020,USA,Florida,"Sombero Key Light, Monroe County",Snorkeling,M,Survived,N,10h30,Bull shark,Sep,0,0,0,0,0,0,0,0,0,0,0,1,1.0,NaN,NaN,NaN,NaN
4,2020,AUSTRALIA,Queensland,Fraser Island,Fishing,M,Survived,N,16h00,"""whitetip shark""",Sep,0,0,0,0,0,0,0,0,0,0,0,1,NaN,1.0,NaN,NaN,NaN
